# Data Analysis

This notebook provides code to visualise and analyze data collected using sensors deployed during the Indaba 2019 maker session

We have deployed the following sensors
1. Ambient Temperature
1. Relative Humidity
1. Soil Moisture Sensor


The sensors are connected to a [Nucleo F446re](https://os.mbed.com/platforms/ST-Nucleo-F446RE/) development board running code contained in this [repo](https://github.com/ciiram/indaba-maker-session-2019).

The data collected from the sensors has been stored on an InfluxDB on your machine. We will connect to this DB and visualise the data. We will use Gaussian processes to interpolate missing values and make predictions from a dataset collected earlier at the Dedan Kimathi University of Technology coffee farm.

## Requirements

1. [pandas](https://pandas.pydata.org/)
1. [matplotlib](https://matplotlib.org/)
1. [GPy](https://sheffieldml.github.io/GPy/) 
1. [influxdb](https://www.influxdata.com/blog/getting-started-python-influxdb/)

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

from pandas import DataFrame, Series
from pandas.io.json import json_normalize

from influxdb import InfluxDBClient
from datetime import datetime, timedelta

## Plots of Ambient Temperature 

We will examine plots of ambient temperature recorded by our device. First we connect to our database to access the data stored.

In [ ]:
client = InfluxDBClient(host='localhost', port=8086)
client.switch_database('indaba_session')

Let us try a few queries

In [ ]:
# Get the last entry

result = client.query('select last("Temperature") from "Indaba Session"')
print(result.raw)

In [ ]:
# Get the entries from the last hour
result = client.query('select * from "Indaba Session" where time > now() - 100h')
print(result.raw)

In [ ]:
result_list = list(result.get_points())

# turn to pandas dataframe
df = pd.DataFrame(result_list)

# make time a datetime object
df[['time']] = df[['time']].apply(pd.to_datetime)

In [ ]:
# Change the Key According to the parameters you measured
%matplotlib inline
plt.figure()
plt.plot(df['time'] + timedelta(hours=3), df['Temperature'], 'bo');
plt.xticks(rotation=45);
plt.ylim([0, 40]);
plt.xlabel('Time');
plt.ylabel('Ambient Temperature (Celcius)');

## Extract and Plot Your Sensor Data

In [ ]:
my_sensor = 'dev_01' # change to your sensor

sensor_groups = df.groupby('sensor')

for name, grp in sensor_groups:
    if name == my_sensor:
        plt.figure()
        plt.plot(grp.time, grp.Temperature, 'bo')
        plt.legend(['Ambient Temperature']);
        plt.xticks(rotation=45);
        plt.ylim([0, 50]);
        plt.xlabel('Time');
        plt.ylabel('Temperature (Celcius)');
        plt.tight_layout()



In [ ]:
# You can write code here to plot other variables
type(grp.time[0])

## Modelling

We will experiment with interpolation using Gaussian process models with various kernels and assess the fits. We will work with the temperature data collected from the coffee farm at Dedan Kimathi University of Technology in September 2018. You can then use the data you collect from your sensors once you have enough data.

In [ ]:
# load the data
ambient_temperature_df = pd.read_csv('data/coffee_farm_temp.csv')
ambient_temperature_df[['Time']] = ambient_temperature_df[['Time']].apply(pd.to_datetime)

# plot it
plt.figure()
plt.plot(ambient_temperature_df['Time'], ambient_temperature_df['Temperature'],'bo')
plt.legend(['Ambient Temperature']);
plt.xticks(rotation=45);
plt.ylim([0, 35]);
plt.xlabel('Time');
plt.ylabel('Temperature (Celcius)');
plt.tight_layout()


## Estimating Missing Values
On the 14th of September, the sensor was down. However we can use Gaussian processes to estimate the temperature on this day. We will use GPy.

In [ ]:
import GPy
import numpy as np
from IPython.display import display

#get the temperature and time
temperature = ambient_temperature_df['Temperature']
time_day = [value.timestamp() for value in ambient_temperature_df['Time']] # get the timestamps
time_day = np.array(time_day)

#normalize to days
time_day = time_day - min(time_day)
time_day /= (24 * 60 * 60)

# get the first 7 days
temperature = temperature[time_day < 7]
time_day = time_day[time_day < 7]


We visualise the data

In [ ]:
plt.figure()
plt.plot(time_day, temperature, 'bo')
plt.ylim([0, 35])
plt.xlabel('Time (days)');
plt.ylabel('Temperature (Celcius)');

We fit a Gaussian process with a radial basis function kernel. (If the optimization fails, simply restart it.)

In [ ]:
#RBF Kernel
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(time_day[:, None],temperature[:, None],kernel)
m.optimize_restarts(num_restarts = 10)
display(m)

Let's plot the fit

In [ ]:
plt.figure()
m.plot();
plt.ylim([-30, 50]);
plt.xlabel('Time (days)');
plt.ylabel('Temperature (Celcius)');

The fit with the RBF kernel does not do a good job of filling in the missing data. In the region without data, the posterior fit is similar to the prior which results in the interpolant passing through zero. 

It is never that cold in Nyeri!!

We now try a periodic kernel on the same data which takes into account the observation that temperature variations are periodic. (If the optimization fails, simply restart it.)

In [ ]:
periodic_kernel = GPy.kern.PeriodicMatern32(input_dim=1, variance=1., lengthscale=1., period=1.)
m = GPy.models.GPRegression(time_day[:, None],temperature[:, None], periodic_kernel)
m.optimize_restarts(num_restarts = 10)

In [ ]:
plt.figure()
m.plot();
plt.ylim([-30, 50]);
plt.xlabel('Time (days)');
plt.ylabel('Temperature (Celcius)');

The fit is still not good. The prior with a zero mean does not fit the data.
We need to add bias. (If the optimization fails, simply restart it.)



In [ ]:
periodic_kernel = GPy.kern.PeriodicMatern32(input_dim=1, variance=1., lengthscale=1., period=1.)
kernel_bias = GPy.kern.Bias(1)

m = GPy.models.GPRegression(time_day[:, None],temperature[:, None], periodic_kernel + kernel_bias)
m.optimize_restarts(num_restarts = 20)

# plot the fit
plt.figure()
m.plot();
plt.ylim([0, 40]);
plt.xlabel('Time (days)');
plt.ylabel('Temperature (Celcius)');

This is a much better fit and the predicted values in the region with missing data is quite reasonable.
